### Задание

Руководитель отдела сопровождения клиентов хочет получить информацию о том, насколько хорошо сотрудники его отдела (аккаунт-менеджеры) справляются с одной из своих основных задач – пролонгацией договоров с клиентами. От аналитика он хочет получить отчет о пролонгациях сотрудников за 2023 год. 

В компании используется два коэффициента пролонгации:
1.	Для проектов пролонгированных в первый месяц – отношении суммы отгрузки  проектов пролонгированных в первый месяц после завершения к сумме отгрузки последнего месяца реализации всех завершившихся в прошлом месяце проектов.
2.	Для проектов, пролонгированных во второй месяц – отношение суммы отгрузки проектов, пролонгированных во второй месяц к сумме отгрузки последнего месяца проектов, не пролонгированных в первый. 

То есть, если нам нужно понять, насколько хорошо менеджер пролонгировал в мае, необходимо посчитать:
1.	Сумму отгрузки проектов, завершившихся в апреле (за апрель) и сумму отгрузки тех проектов завершившихся в апреле, у которых есть отгрузка в мае (за май). Коэффициент – отношение второй суммы к первой. 
2.	Сумму проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март) и сумму отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле но есть в мае (за май). Коэффициент – отношение второй суммы к первой. 

Имеются два набора данных:

1.	prolongations.csv
	id – id проекта
	month – последний месяц реализации проекта
	AM – ФИО ответственного аккаунт-менеджера (данные первичны по отношению к financial_data)

2.	financial_data.csv:
	id – id проекта
	Причина дубля – причина, почему строки с одним и тем же id встречаются несколько раз
	Колонки с названием месяца – сумма отгрузки проекта в данный месяц 
	Account – ФИО ответственного аккаунт-менеджера

Необходимо: 
1.	Рассчитать коэффициенты пролонгации для каждого менеджера и для всего отдела в целом
a.	за каждый месяц
b.	за год
2.	Сформировать аналитический отчет в гугл-таблицах или excel, на основании которого руководитель отдела будет принимать управленческие решения (обязательно присутствие коэффициентов пролонгации, в остальном отчет можно дополнить на свое усмотрение – дополнительные метрики и визуализация приветствуются)

В результате ждем 2 файла:
1.	Код с подробными пояснениями логики расчетов в Google Colab или Jupiter Notebook
2.	Аналитический отчет для руководителя отдела (в excel/google sheets)


  В данной колонке могут также встречаться такие значения: ‘в ноль’ –  отгрузка проекта в данном месяце равна 0, значит для коэффициента пролонгации нужно взять отгрузку предыдущего месяца (только если все части оплаты равны 0); ‘стоп’ – проект закончился до истечения срока договора, если у проекта есть “стоп” в последний месяц реализации или ранее, то такой проект исключаем из пролонгаций; ‘end’ – аналогично ‘стоп’


In [ ]:
import pandas as pd
# Читаем датасеты с помощью Pandas
df_prol = pd.read_csv('prolongations.csv', encoding='utf-8')
df_fina = pd.read_csv('financial_data.csv',encoding='utf-8')

# Сначала обрабатываем датасет с финансами, так как нам надо разобраться с "стоп", "end", "в ноль", 
# а также убрать неразрывный пробел, заменить нуллы на 0 и перевести все в int

# Замещаем нулевые значения строковым 0

df_fina = df_fina.fillna('0')

# Убираем лишний пробел, помеченный как \xa0
for i in df_fina:
    if df_fina[i].dtype == 'object':
        df_fina[i] = df_fina[i].str.replace('\xa0','') # Убираем неразрывный пробел из чисел
        df_fina[i] = df_fina[i].str.replace(',','.') # Заменяем запятую на точку для того, чтобы потом перевести все в числа
        df_fina[i] = df_fina[i].str.replace('end','стоп') # Так как "end" и "стоп" обозначают одно и то же, оставим только "стоп"
        df_fina[i] = df_fina[i].str.replace('в ноль','0.0001') # Помечаем числом "в ноль"


dict_month_stop = dict()
list_month_stop = []
list_i_df_fina = [i for i in df_fina if '20' in i] # Название столбцов, где должны быть суммы
list_fina = [i.lower() for i in list_i_df_fina]
for i in list_i_df_fina:
    dict_month_stop[i.lower()] = list(set(df_fina[df_fina[i]=='стоп'].id))
    list_month_stop.extend(list(df_fina[df_fina[i]=='стоп'].id))
list_month_stop = list(set(list_month_stop))
list_month_stop.sort()


# Выделяем индексы проектов, которые не будут считаться в прологнации ("стоп" в тот же месяц или раньше)
list_fina = [i.lower() for i in list_i_df_fina]
list_del_index = []
list_not_del_index = []
for key in dict_month_stop.keys():
    for i in dict_month_stop[key]:
        month_list = df_prol[df_prol.id == i]['month'].values
        if list_fina.index(key) <= list_fina.index(month_list[0]):
            list_del_index.append(i) # Это те, что будут исключены из df
        else:
            list_not_del_index.append(i)

# Выделяем те проекты, в которых "стоп" появился раньше или во время месяца завершения проекта
l_del_ind_fina = []
l_del_ind_prol = []
for i in list_del_index:
    l_del_ind_fina.extend(list(df_fina[df_fina.id == i].index))
    l_del_ind_prol.extend(list(df_prol[df_prol.id == i].index))

df_fina = df_fina.drop(l_del_ind_fina, axis=0)
df_prol = df_prol.drop(l_del_ind_prol, axis=0)

for i in df_fina:
    if df_fina[i].dtype == 'object':
        df_fina[i] = df_fina[i].str.replace('стоп','0') 

df_fina = df_fina.astype({i:'float' for i in list_i_df_fina})

df_f = df_fina.groupby('id').agg({i:'sum' for i in list_i_df_fina})

merged_df = pd.merge(df_prol, df_f, on=['id', 'id'])

In [267]:
# Для того, чтобы заменить "в ноль" на значения
for i in range(len(merged_df)):
    for num,j in enumerate(list_i_df_fina):
        if merged_df[i:i+1][j].values[0] == 0.0001:
            print(merged_df[i:i+1][list_i_df_fina[num-1]].values[0])
            print('AAA"')
            merged_df.loc[i:i+1,j] = merged_df[i:i+1][list_i_df_fina[num-1]].values[0]

In [268]:
merged_df

,id,month,AM,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024
0,42,ноябрь 2022,Васильев Артем Александрович,36220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,453,ноябрь 2022,Васильев Артем Александрович,0.0,39245.0,44320.0,177635.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,548,ноябрь 2022,Михайлов Андрей Сергеевич,0.0,674000.0,674000.0,674000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,87,ноябрь 2022,Соколова Анастасия Викторовна,70050.0,0.0,73380.0,83480.0,89300.0,89300.0,78605.0,72485.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,429,ноябрь 2022,Соколова Анастасия Викторовна,30280.0,35580.0,35830.0,42830.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,955,декабрь 2023,Смирнова Ольга Владимировна,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9300.0,9300.0,13200.0,17400.0,0.0,0.0
436,1004,декабрь 2023,без А/М,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34000.0,0.0,0.0,0.0
437,281,декабрь 2023,Соколова Анастасия Викторовна,98000.0,93250.0,82800.0,109000.0,125800.0,91750.0,94700.0,129740.0,117730.0,115860.0,160770.0,142490.0,99125.0,74350.0,105775.0,92065.0
438,785,декабрь 2023,Соколова Анастасия Викторовна,0.0,0.0,0.0,5306.6,12898.1,5287.0,10180.0,8600.0,3860.0,8600.0,700.0,700.0,700.0,700.0,0.0,0.0


In [269]:
def get_month_k1_k2(d,manager,month_prolong, month_before = 0, month_before_before = 0):
    if month_before == 0:
        return [manager,month_prolong,0,0], [month_prolong,0,0,0,0]
    df = d[(d.AM==manager) & (d.month == month_before.lower())]
    
    # Первый коэффициент пролонгации менеджеров
    if month_before == 0:
        k1 = 0
    else:
        part_1,part_2 = 0,0
        for i in range(len(df)):
            if df[i:i+1][month_prolong].values[0]>0:
                part_1 += df[i:i+1][month_before].values[0]
            part_2 += df[i:i+1][month_before].values[0]
        if part_2 == 0:
            k1 = 0
        else:
            k1 = part_1/part_2
    
    # Второй коэффициент пролонгации менеджеров для мая
    part_3, part_4 = 0,0
    if month_before == 0 or month_before_before == 0:
        k2 = 0
    else:
        for i in range(len(df)):
            if df[i:i+1][month_before].values[0]!=0:
                if df[i:i+1][month_prolong].values[0]>0:
                    part_3 += df[i:i+1][month_before_before].values[0]
                part_4 += df[i:i+1][month_before_before].values[0]
        if part_4 == 0:
            k2 = 0
        else:
            k2 = part_3/part_4
    return [manager,month_prolong,k1,k2], [month_prolong,part_1,part_2,part_3,part_4]

In [273]:
# Получаем коэффициенты для менеджеров
dik = []
for_count_list = []
for i in df_prol['AM'].unique():
    for num, item in enumerate(list_i_df_fina):
        if num == 0:
            result, for_count = get_month_k1_k2(merged_df,i,list_i_df_fina[num], 0, 0)
        elif num == 1:
            result, for_count = get_month_k1_k2(merged_df,i,list_i_df_fina[num], list_i_df_fina[num-1], 0)
        else:
            result, for_count = get_month_k1_k2(merged_df,i,list_i_df_fina[num], list_i_df_fina[num-1], list_i_df_fina[num-2])
        dik.append(result)
        for_count_list.append(for_count)
duk = pd.DataFrame(dik, columns=['Аккаунт менеджер','Месяц Год','Коэффициент пролонгации 1','Коэффициент пролонгации 2'])
duk.to_csv('AM.csv')

In [274]:
# Посчитаем пролонгацию по месцам для всего отдела
department_k1_k2 = []
for i in list_i_df_fina:
    part_1, part_2, part_3, part_4 = 0,0,0,0
    for j in for_count_list:
        if j[0] == i:
            part_1 += j[1]
            part_2 += j[2]
            part_3 += j[3]
            part_4 += j[4]
        if part_2 == 0 and part_4 != 0:
            k1 = 0
            k2 = part_3/part_4
        elif part_4 == 0 and part_2 != 0:
            k1 = part_1/part_2
            k2 = 0
        elif part_2 == 0 and part_4 == 0:
            k1 = 0
            k2 = 0
        else:
            k1 = part_1/part_2
            k2 = part_3/part_4
    department_k1_k2.append([i, k1, k2, part_1, part_2, part_3, part_4])
pduuk = pd.DataFrame(department_k1_k2, columns=['Месяц Год','Коэффициент пролонгации 1','Коэффициент пролонгации 2', 'k1_1','k1_2','k2_1','k2_2'])
pduuk.to_csv('department.csv')

In [275]:
# За год
part_1 = sum(pduuk['k1_1'])
part_2 = sum(pduuk['k1_2'])
part_3 = sum(pduuk['k2_1'])
part_4 = sum(pduuk['k2_2'])
k1 = part_1/part_2
k2 = part_3/part_4
year_sum = pd.DataFrame(['year',k1,k2])
year_sum.to_csv('year.csv')